In [ ]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 28.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Open the folder where you saved your prepapred datasets from the first exercise
# You might need to mount your Google Drive first
%cd /content/drive/MyDrive/NLP_Project/it_es/it_to_es/nmt/
!ls

/content/drive/MyDrive/NLP_Project/it_es/it_to_es/nmt
config.yaml
Europarl.es-it.es
Europarl.es-it.es-filtered.es
Europarl.es-it.es-filtered.es.subword
Europarl.es-it.es-filtered.es.subword.dev
Europarl.es-it.es-filtered.es.subword.test
Europarl.es-it.es-filtered.es.subword.test.desubword
Europarl.es-it.es-filtered.es.subword.train
Europarl.es-it.it
Europarl.es-it.it-filtered.it
Europarl.es-it.it-filtered.it.subword
Europarl.es-it.it-filtered.it.subword.dev
Europarl.es-it.it-filtered.it.subword.test
Europarl.es-it.it-filtered.it.subword.train
Europarl.es-it.xml
Europarl.es.translated
Europarl.es.translated2
Europarl.es.translated2.desubword
Europarl.es.translated.desubword
LICENSE
MT-Preparation
README
run
source.model
source.vocab
target.model
target.vocab
train.log


In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: Europarl.es-it.it-filtered.it.subword.train
        path_tgt: Europarl.es-it.es-filtered.es.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: Europarl.es-it.it-filtered.it.subword.dev
        path_tgt: Europarl.es-it.es-filtered.es.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 3

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 5000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 25000

# Default: 10000 - Run validation after n steps
valid_steps: 500

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 2000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 3
dec_layers: 3
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

2


In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-18 22:14:04,745 INFO] Counter vocab from -1 samples.
[2023-06-18 22:14:04,745 INFO] n_sample=-1: Build vocab on full datasets.
[2023-06-18 22:16:00,220 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=526)

[2023-06-18 22:16:00,232 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=478)

[2023-06-18 22:16:00,418 INFO] Counters src: 50312
[2023-06-18 22:16:00,418 INFO] Counters tgt: 50383


In [ ]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-80560d1c-ea63-75b2-f6b4-070c56c2379e)


In [ ]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14998.8125 out of: 15101.8125


In [ ]:
# Train the NMT model
!onmt_train -config config.yaml

[2023-06-25 07:22:28,000 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-25 07:22:28,258 INFO] Parsed 2 corpora from -data.
[2023-06-25 07:22:28,258 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-25 07:22:29,364 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁di', '.', "'", '▁e', '▁che']
[2023-06-25 07:22:29,365 INFO] The decoder start token is: <s>
[2023-06-25 07:22:29,365 INFO] Building model...
[2023-06-25 07:22:30,730 INFO] Switching model to float32 for amp/apex_amp
[2023-06-25 07:22:30,731 INFO] Non quantized layer compute is fp16
[2023-06-25 07:22:34,982 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50000, 512, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): 

In [ ]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate -model models/model.fren_step_5000.pt -src Europarl.es-it.it-filtered.it.subword.test -output Europarl.es.translated -gpu 0 -min_length 1

[2023-06-19 09:26:01,081 INFO] Loading checkpoint from models/model.fren_step_5000.pt
[2023-06-19 09:28:01,770 INFO] PRED SCORE: -0.4316, PRED PPL: 1.54 NB SENTENCES: 4000


In [ ]:
!onmt_translate -model models/model.fren_step_15000.pt -src Europarl.es-it.it-filtered.it.subword.test -output Europarl.es.translated2 -gpu 0 -min_length 1

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Intel MKL FATAL ERROR: Cannot load /usr/local/lib/python3.10/dist-packages/torch/lib/libtorch_cpu.so.


In [ ]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.es.translated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00
Done desubwording! Output: Europarl.es.translated.desubword


In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.es.translated2

Done desubwording! Output: Europarl.es.translated2.desubword


In [ ]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation,
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py target.model Europarl.es-it.es-filtered.es.subword.test

Done desubwording! Output: Europarl.es-it.es-filtered.es.subword.test.desubword


In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-06-19 09:36:18--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2023-06-19 09:36:19 (37.3 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
# Install sacrebleu
!pip3 install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py Europarl.es-it.es-filtered.es.subword.test.desubword Europarl.es.translated.desubword

Reference 1st sentence: ¡Pues bien, allá vamos!
MTed 1st sentence: ¡Desde entonces!
BLEU:  35.01755976768619


In [ ]:
!python3 compute-bleu.py Europarl.es-it.es-filtered.es.subword.test.desubword Europarl.es.translated2.desubword

Reference 1st sentence: ¡Pues bien, allá vamos!
MTed 1st sentence: ¡Adelante, entonces!
BLEU:  36.61720026445025
